In [17]:
import tensorflow as tf
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt

class neuralNetwork:
    
    def __init__(self, _numLayer, _currNumUnit):
        self.numLayer = _numLayer
        self.currNumUnit = _currNumUnit
        

    def buildLayer(self, _inputTensor, _numUnits):
        '''
        input:
            _inputTensor S = theta(Xprev)from the prev layer
            _numUnits is the num of neurons in this layer
    
        intermediate xavierInit:
            W is initialized as Xavier
            W is input.shape[1] by num units
    
        output:
        weighted sum of inputs
    
        '''
    
        # zero mean independent Gaussians whose variance is 3/(#input + #outputs)

    
        dim1 = _inputTensor.get_shape().as_list()[1]
        #Xavier
        #W = tf.get_variable("W", shape = [dim1, _numUnits], initializer=tf.contrib.layers.xavier_initializer())
        variance = 3.0 / (dim1 + _numUnits)
        W = tf.variable(tf.truncated_normal(shape = [dim1, _numUnits],  stddev = math.sqrt(variance)))
        b = tf.Variable(0.0, name='biases')
        #print("W", W)
        X = tf.matmul(tf.transpose(W), tf.cast(_inputTensor, dtype = tf.float_32))
        return X
    

    def buildNet(self, _data, _numUnits):
        '''
        input: 
            number of hidden units
            data
        output:
            predicted labels
        '''
        S_prev = tf.convert_to_tensor(_data)
        for i in self.currNumUnit:              
            S_curr = buildLayer(S_prev, numUnits)
            #S is passed to relu already
            S_prev = S_curr
                      
        #calculate error
        #gradient descent
    
        

    
    
class loadData:
    def __init__(self):
        self.flatten = True
        self.addOnes = False
        
        self.data_path = '/Users/vikuo/Documents/GitHub/ece521/assi/A3/notMNIST.npz'
    def arrFlatten(self, arr):
        '''
        type np array
        '''
        dataDim1, dum1, dum2 = arr.shape
        dum_sq = dum1 * dum2
        arr = np.reshape(arr, [ dataDim1 ,dum_sq ])
        return arr   
    def convertTarget(self, targetValues):
        numClasses = np.max(targetValues) + 1
        return np.eye(numClasses)[targetValues]

    def loadNotMnits(self):
        with np.load(self.data_path) as data:
            Data, Target = data ["images"], data["labels"]
            np.random.seed(521)
            randIndx = np.arange(len(Data))
            np.random.shuffle(randIndx)
            Data = Data[randIndx]/255.
            
            if self.flatten:
                Data = self.arrFlatten(Data)
            
            Target = Target[randIndx]
            trainData, trainTarget = Data[:15000], Target[:15000]
            validData, validTarget = Data[15000:16000], Target[15000:16000]
            testData, testTarget = Data[16000:], Target[16000:]
            
            trainTarget = self.convertTarget(trainTarget)
            validTarget = self.convertTarget(validTarget)
            testTarget = self.convertTarget(testTarget)
 
        return trainData, trainTarget, validData, validTarget, testData, testTarget

class BatchSampler(object):
    '''
    A (very) simple wrapper to randomly sample batches without replacement.
    '''
    
    def __init__(self, data, targets, batch_size):
        self.num_points = data.shape[0]
        self.features = data.shape[1]
        self.data = data
        self.targets = targets
        self.batch_size = batch_size
        self.indices = np.arange(self.num_points)

    def random_batch_indices(self, m=None):
        if m is None:
            indices = np.random.choice(self.indices, self.batch_size, replace=False)
        else:
            indices = np.random.choice(self.indices, m, replace=False)
        return indices 

    def get_batch(self, m=None):
        '''
        Get a random batch without replacement from the dataset.
        If m is given the batch will be of size m. 
        Otherwise will default to the class initialized value.
        '''
        indices = self.random_batch_indices(m)
        X_batch = np.take(self.data, indices, 0)
        y_batch = self.targets[indices]
        return X_batch, y_batch

def hyperParamGen():
    #random sampes using numpy
    logLearningRate = np.random.uniform(-7.5, -4.5)
    print("logLearningRate", logLearningRate)
    learningRate = np.exp(logLearningRate)
    print("learningRate", learningRate)
    numLayers = np.random.random_integers(1, 5)
    print("numLayers", numLayers)
    numHiddenUnits = np.random.random_integers(100, 500)
    print("numHiddenUnits", numHiddenUnits)
    logWeightedDecay = np.random.uniform(-9, -6)
    print("logWeightedDecay", logWeightedDecay)
    weightedDecay = np.exp(logWeightedDecay)
    print("weightedDecay", weightedDecay)
    dropOut = np.random.random_integers(0, 1) # 0 or 1
    print("dropOut", dropOut)
    
    return learningRate, numLayers, numHiddenUnits, weightedDecay, dropOut
    
if __name__ == '__main__':  


    #tf.reset_default_graph()
    #init = tf.global_variables_initializer()
    #sess = tf.InteractiveSession()
    #sess.run(init)
    
                      


    

logLearningRate -6.142641725270684
learningRate 0.0021492384148
numLayers 3
numHiddenUnits 202
logWeightedDecay -6.221198129637846
weightedDecay 0.00198686327084
dropOut 0


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:149: DeprecationWarning: This function is deprecated. Please call randint(1, 5 + 1) instead
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:151: DeprecationWarning: This function is deprecated. Please call randint(100, 500 + 1) instead
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:157: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
